# 네이버쇼핑

## #01.필요한 모듈 참조

In [5]:
import requests                       # -> 웹페이지 요청 모듈
import os                             # -> 날짜 처리 모듈
import urllib                         # -> 파이썬 기본 모듈
import datetime as dt                 # -> 날짜 처리 모듈
from bs4 import BeautifulSoup         # -> 웹페이지 소스코드 분석 모듈
from pandas import DataFrame          # -> 데이터프레임

## #02. 수집 준비

1) 접속을 수행하기 위한 session 객체 생성

In [2]:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"

session = requests.Session()
session.headers.update({'User-agent':user_agent, 'referer':None})

2) 접근할 페이지주소(네이버 쇼핑에 대한 검색 결과 주소)

필요한 변수 지정

In [3]:
keyword = "카메라"   # 검색어
max_page = 5         #가져올 최대 페이지

URLEncoding을 적용한 접속 주소 만들기

In [9]:
page_url = "https://search.shopping.naver.com/search/all.nhn"
params = {"originquery":keyword, "pagingIndex" : 1,"pagingSize":40, "viewType" : "list", "sort":"rel","frm":"NVSHATC", "query":keyword}

## #03. 반복을 통한 페이지 컨텐츠 수집

In [13]:
# 수집 결과를 누적할 빈 리스트
data_list = []

for page_no in range(1, max_page+1):
    params['pagingIndex'] = page_no
    query = urllib.parse.urlencode(params)
    content_url = page_url + "?" + query
    print(content_url)
    
    r=session.get(content_url)
    
    if r.status_code != 200:
        print("%d 에러가 발생했습니다." % r.status_code)
        continue
    
    r.encoding = "utf-8"
    soup = BeautifulSoup(r.text, 'html.parser')
    
    info = soup.select(".info")
    
    #print(info)
    
    # 검색된 상품목록 영역 수 만큼 반복
    for item in info:
        #하나의 상품 정보를 담기 위한 빈 딕셔너리
        item_dict = {}
        
        #상품명 추축
        # -> class 이름으로 상품명 영역 추출
        title_list = item.select('.tit > a')
        # -> 상품명은 하나만 존재하므로 0번째에 직접 접근하여 텍스트를 추출하여 빈 딕셔너리에 추가
        item_dict['제품명'] = title_list[0].text.strip()
        
        #가격 추출
        #-> class 이름으로 가격 영역 추출
        price_list = item.select('.num')
        #-> 가격은 하나만 존재하므로 0번째에 직접 접근하여 텍스트 추출
        price = price_list[0].text.strip()
        #-> 상품 가격에 포함된 콤마를 빈 문자열로 변경
        price = price.replace(",","")
        #-> 정수형으로 변환된 결과를 빈 딕셔너리에 추가
        item_dict['가격']=int(price)
        
        # 노트북 스팩 영역 추출
        #-> class 이름으로 접근하여 그 하위의 링크 추출
        spec_list = item.select('.detail > a')
        
        #-> 제품별로 스펙 항목이 여러개 이므로 추출된 결과만큼 반복
        for v in spec_list:
            v = v.text.strip() #-> 텍스트 추출
            tmp = v.split(":") #-> 콜론으로 분리(ex:디스플레이:1920X1080)
            
            if len(tmp) == 2:  # -> 길이가 2라면 key와 value로 분리하여 딕셔너리에 추가
                key = tmp[0].strip()
                value = tmp[1].strip()
                item_dict[key] = value
        #개별 상품 정보를 빈 리스트에 누적
        data_list.append(item_dict)

data_list

https://search.shopping.naver.com/search/all.nhn?originquery=%EC%B9%B4%EB%A9%94%EB%9D%BC&pagingIndex=1&pagingSize=40&viewType=list&sort=rel&frm=NVSHATC&query=%EC%B9%B4%EB%A9%94%EB%9D%BC
https://search.shopping.naver.com/search/all.nhn?originquery=%EC%B9%B4%EB%A9%94%EB%9D%BC&pagingIndex=2&pagingSize=40&viewType=list&sort=rel&frm=NVSHATC&query=%EC%B9%B4%EB%A9%94%EB%9D%BC
https://search.shopping.naver.com/search/all.nhn?originquery=%EC%B9%B4%EB%A9%94%EB%9D%BC&pagingIndex=3&pagingSize=40&viewType=list&sort=rel&frm=NVSHATC&query=%EC%B9%B4%EB%A9%94%EB%9D%BC
https://search.shopping.naver.com/search/all.nhn?originquery=%EC%B9%B4%EB%A9%94%EB%9D%BC&pagingIndex=4&pagingSize=40&viewType=list&sort=rel&frm=NVSHATC&query=%EC%B9%B4%EB%A9%94%EB%9D%BC
https://search.shopping.naver.com/search/all.nhn?originquery=%EC%B9%B4%EB%A9%94%EB%9D%BC&pagingIndex=5&pagingSize=40&viewType=list&sort=rel&frm=NVSHATC&query=%EC%B9%B4%EB%A9%94%EB%9D%BC


[{'제품명': '후지필름 XF10',
  '가격': 539200,
  '최대조리개': 'F2.8',
  '접사거리': '10cm',
  '최대ISO감도': 'ISO 51200',
  '무게': '278.9g',
  '최소셔터스피드': '1/4000초',
  '최대연속촬영속도': '초당6매',
  '저장매체': 'SD',
  '부가기능': '얼굴인식AF'},
 {'제품명': '후지필름 X-A7',
  '가격': 885480,
  '품목': '미러리스',
  '용도별': '입문자용',
  '유효화소': '2420만화소',
  '동영상 해상도': 'UHD(4K)',
  '저장매체': 'SD카드',
  '프레임': '30프레임',
  '형태': '회전(셀카)',
  '스피드': '셔터(1/4000초)',
  '연사': '초당6장',
  '초점방식': '위상차AF',
  '초점영역': '256개',
  '플래시': '팝업플래시',
  '화면크기': '3.5인치',
  '화면화소': '276만화소',
  '화면': '터치',
  '무선기능': '블루투스',
  '촬영기능': '필름시뮬레이션',
  'ISO': '200~12800',
  '확장': '100~51200',
  '최대촬영': '약270장(사진),약55분(동영상)',
  '이미지센서 부가기능': '초음파방진필터',
  '무게(배터리제외)': '271g',
  '무게': '320g',
  '인터페이스': 'HDMI'},
 {'제품명': '후지필름 X-T100',
  '가격': 519040,
  '품목': '미러리스',
  '용도별': '중급자용',
  '유효화소': '2420만화소',
  '부가기능': '먼지제거',
  '뷰파인더': '전자식',
  '최대ISO감도': 'ISO 12800',
  '동영상 해상도': 'UHD(4K)',
  '최대연속촬영속도': '초당6매',
  '동영상프레임': '15프레임',
  '저장매체': 'SD',
  '프레임': '15프레임',
  '형태': '셀카',
  '스피드': 

In [14]:
df = DataFrame(data_list)
df

,제품명,가격,최대조리개,접사거리,최대ISO감도,무게,최소셔터스피드,최대연속촬영속도,저장매체,부가기능,...,MOS,셔터기능,전자식뷰파인더종류,CCD,줌기능,가이드,배터리용량,동영상 촬영시간,전원,바디재질
0,후지필름 XF10,539200,F2.8,10cm,ISO 51200,278.9g,1/4000초,초당6매,SD,얼굴인식AF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,후지필름 X-A7,885480,NaN,NaN,NaN,320g,NaN,NaN,SD카드,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,후지필름 X-T100,519040,NaN,NaN,ISO 12800,448g,NaN,초당6매,SD,먼지제거,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,폴라로이드 SNAP TOUCH,252750,NaN,NaN,NaN,256g,NaN,NaN,MICRO SD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,소니 알파 A5100,315600,NaN,NaN,ISO 25600,283g,1/4000초,NaN,메모리스틱ProDuo,먼지제거,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,삼성전자 WB250F,236030,F3.2,5cm,ISO 3200,184g,1/2000초,8매,SD,손떨림방지,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,파나소닉 DMC-GX85,515060,NaN,NaN,ISO 25600,426g,NaN,6매,SD,로우패스필터제거,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204,캐논 호환 EOS M10 EF-M 15-45mm 렌즈 전용필터,29870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205,소니 A5100 16-50mm + SD16G 메모리 + U11 가방,524590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df.to_excel(keyword+".xlsx")